In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
# Activate mypy type checking
%load_ext nb_mypy

In [ ]:
import nbimporter # Activate import functionality for notebooks

# Chess-AI Statistiken

Dieses Notebook lässt jede AI Version gegeneinander spielen und exportiert die Ergebnisse als CSV-Datei.

Importierte Versionen:
- `AI-Base-Class.ipynb`: Die abstrakte Basisklasse für alle AI-Versionen, welche das Eröffnungs- und Endspiel implementiert.
- `Exercise01AI`: Die erste Version der AI, welche im Mittelspiel zufällige Züge auswählt.
- `Exercise02AI`: Die zweite Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und dem einfachen Materialwert auswählt.
- `Exercise03AI`: Die dritte Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und der `simple evaluation function` auswählt.
- `Exercise04AI`: Die vierte Version der AI, welche im Vergleich zur dritten Version mithilfe von Memoisierung beschleunigt wurde.
- `StockfishPlayer`: Ein Spieler, welcher durch die Stockfish Engine gesteuert wird.

In [ ]:
import os

# Cleanup game folder (delete all files)
for filename in os.listdir("../games"):
    os.remove(f"../games/{filename}")

In [ ]:
from Exercise01AI import Exercise01AI
from Exercise02AI import Exercise02AI
from Exercise03AI import Exercise03AI
from Exercise04AI import Exercise04AI
from StockfishPlayer import StockfishPlayer
from Main import run_games 

In [ ]:
# Build relation set of all players
players = [Exercise01AI, Exercise02AI, Exercise03AI, Exercise04AI, StockfishPlayer]
player_pairs = {(white, black) for white in players for black in players}
player_pairs

In [ ]:
%%time
# Let all player pairs play
for i, players in enumerate(player_pairs):
    print(f"Playing game {i+1} of {len(player_pairs)}")
    run_games(players[0]("White"), players[1]("Black"), repetitions=1, seed=3)

In [ ]:
# Collect results from .pgn files
import chess.pgn

results_map_white = {"1-0": "Won", "0-1": "Lost", "1/2-1/2": "Remis"}
games_map: dict[str, list] = {}

for filename in os.listdir("../games"):
    if not filename.endswith(".pgn"):
        continue
    with open(os.path.join("..", "games", filename)) as pgn:
        pgn_game = chess.pgn.read_game(pgn)
    if not pgn_game:
        continue

    white_view = {
        "date": pgn_game.headers["Date"],
        "round": pgn_game.headers["Round"],
        "opponent": pgn_game.headers["Black"],
        "result": results_map_white[pgn_game.headers["Result"]],
        "duration": pgn_game.headers["Duration"],
        "seed": pgn_game.headers["Seed"],
        "state_changes": {node.comment: node.ply() for node in pgn_game.mainline() if node.comment},
        "total_moves": pgn_game.end().ply(),
        "filename": filename
    }
    entry = games_map.get(pgn_game.headers["White"], [])
    entry.append(white_view)
    games_map[pgn_game.headers["White"]] = entry

In [ ]:
import csv

# Export to csv
with open(os.path.join("..", "games", "statistics.csv"), 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow([
        "Player", 
        "Opponent", 
        "Result", 
        "Moves", 
        "Duration", 
        "Moves to middle game", 
        "Moves to end game", 
        "Date", 
        "Round", 
        "Seed",
        "File"
    ])
    keys = sorted(games_map)
    for player in sorted(games_map):
        games = sorted(games_map[player], key=lambda g: g["opponent"])
        for game in games:
            csv_writer.writerow([
                player, 
                game["opponent"], 
                game["result"], 
                game["total_moves"], 
                game["duration"].replace(".", ","), 
                game["state_changes"].get("State.MIDDLE_GAME", "-"), 
                game["state_changes"].get("State.END_GAME", "-"), 
                game["date"],
                game["round"].replace("/", " of "),
                game["seed"],
                game["filename"],
            ])
        csv_writer.writerow([])